# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter 
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secret import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
response = requests.get(api_url)
print(response.status_code)
data = response.json()[0]
print(data)

200
{'avgTotalVolume': 64867409, 'calculationPrice': 'close', 'change': 0.9, 'changePercent': 0.0055, 'close': 164.66, 'closeSource': 'official', 'closeTime': 1680811200252, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 164.63, 'delayedPriceTime': 1680811181490, 'extendedChange': -0.09, 'extendedChangePercent': -0.00055, 'extendedPrice': 164.57, 'extendedPriceTime': 1680825599748, 'high': 164.66, 'highSource': 'Close', 'highTime': 1680811200252, 'iexAskPrice': None, 'iexAskSize': None, 'iexBidPrice': None, 'iexBidSize': None, 'iexClose': 164.66, 'iexCloseTime': 1680811199902, 'iexLastUpdated': None, 'iexMarketPercent': None, 'iexOpen': 162.37, 'iexOpenTime': 1680787801037, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexVolume': None, 'lastTradeTime': 1680811199940, 'latestPrice': 164.66, 'latestSource': 'Close', 'latestTime': 'April 6, 2023', 'latestUpdate': 1680811200252, 'latestVolume': 45390123, 'low': 164.66, 'lowSource': 'Close', 'lowTime': 1680811200000, 

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
data['latestPrice']

164.66

In [6]:
data['marketCap']

2605241628360

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [9]:
my_columns = ['Ticker', 'Stock Price','Market Capitalization', 'Number Of Shares to Buy']

final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number Of Shares to Buy


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [43]:
import warnings
warnings.filterwarnings("ignore")
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in stocks['Ticker']:
    api_url = f'https://api.iex.cloud/v1/data/core/quote/{symbol}?token={IEX_CLOUD_API_TOKEN}'
    response = requests.get(api_url)
    data = response.json()
    final_dataframe = final_dataframe.append(data, ignore_index=True)
    final_dataframe = final_dataframe[['symbol', 'latestPrice', 'marketCap']]
    final_dataframe['Number of Shares to Buy'] = 'N/A'

In [48]:
final_dataframe.sort_values(by = "symbol", ascending = True).head(105)

,symbol,latestPrice,marketCap,Number of Shares to Buy
0,A,138.03,4.081573e+10,N/A
1,AAL,14.01,9.145959e+09,N/A
2,AAP,120.69,7.153753e+09,N/A
3,AAPL,164.66,2.605242e+12,N/A
4,ABBV,161.55,2.849971e+11,N/A
...,...,...,...,...
100,CME,195.10,7.018522e+10,N/A
101,CMG,1685.21,4.654861e+10,N/A
102,CMI,220.33,3.118545e+10,N/A
103,CMS,62.31,1.817281e+10,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [10]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [11]:
import warnings
warnings.filterwarnings("ignore")

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    #print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/v1/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    response = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'HFC' or symbol == 'VIAC' or symbol == 'WLTW' or symbol == 'DISCA':
            continue
        else:
            final_dataframe =  final_dataframe.append(
            pd.Series(
            [
                symbol,
                response[symbol]['quote']['latestPrice'],
                response[symbol]['quote']['marketCap'],
                'N/A'
            ],
            index = my_columns),
            ignore_index = True
        )


In [12]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number Of Shares to Buy
0,A,138.03,40815726770,N/A
1,AAL,14.01,9145959039,N/A
2,AAP,120.69,7153752629,N/A
3,AAPL,164.66,2605241628360,N/A
4,ABBV,161.55,284997105690,N/A
...,...,...,...,...
496,YUM,132.10,37002248702,N/A
497,ZBH,129.17,26993979668,N/A
498,ZBRA,300.26,15433166129,N/A
499,ZION,29.39,4352631315,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [20]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")
    val = float(portfolio_size)

Enter the value of your portfolio: 5000000


In [21]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i,'Stock Price'])
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number Of Shares to Buy
0,A,138.03,40815726770,72
1,AAL,14.01,9145959039,712
2,AAP,120.69,7153752629,82
3,AAPL,164.66,2605241628360,60
4,ABBV,161.55,284997105690,61
...,...,...,...,...
496,YUM,132.10,37002248702,75
497,ZBH,129.17,26993979668,77
498,ZBRA,300.26,15433166129,33
499,ZION,29.39,4352631315,339


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [22]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, sheet_name='Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [106]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [103]:

# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
# writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [107]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Stock Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [108]:
writer.save()